In [ ]:
from datasets import Dataset,load_dataset
import datasets

In [ ]:
dataset = load_dataset('banking77')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3080 [00:00<?, ? examples/s]

In [ ]:
for split,dataset in dataset.items():
  dataset.to_csv(f"my-dataset-{split}.csv",index=None)

Creating CSV from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
import pandas as pd

df=pd.read_csv("my-dataset-train.csv")

In [ ]:
from datasets import Dataset
df['encoded_text'] = df['label'].astype('category').cat.codes
data_texts = df['text'].to_list()
data_labels = df['encoded_text'].to_list()
dataset = Dataset.from_dict({"text": data_texts, "label": data_labels})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_text(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset = dataset.map(tokenize_text, batched=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/10003 [00:00<?, ? examples/s]

In [ ]:
train_dataset = dataset.shuffle(seed=42).train_test_split(test_size=0.1)["train"]
val_dataset = dataset.shuffle(seed=42).train_test_split(test_size=0.1)["test"]

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np
from datasets import load_metric
f1_metric=load_metric("f1")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    model_accuracy = accuracy.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels,average="macro")
    return {
        "accuracy":model_accuracy,
        "f1":f1
    }

<ipython-input-15-914fa5dd634c>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  f1_metric=load_metric("f1")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=77)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 9002
})

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="model_distill",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    eval_steps=500,
    logging_dir="logs",
    logging_steps=500,
    save_strategy="steps"


)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    #data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,F1
500,3.487300,2.464031,{'accuracy': 0.6743256743256744},{'f1': 0.5962740008490768}
1000,2.178000,1.737480,{'accuracy': 0.7732267732267732},{'f1': 0.7229582983680111}


Trainer is attempting to log a value of "{'accuracy': 0.6743256743256744}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.5962740008490768}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.7732267732267732}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.7229582983680111}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=1126, training_loss=2.72463991968077, metrics={'train_runtime': 862.9548, 'train_samples_per_second': 20.863, 'train_steps_per_second': 1.305, 'total_flos': 2388132950913024.0, 'train_loss': 2.72463991968077, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

Trainer is attempting to log a value of "{'accuracy': 0.7732267732267732}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.7229582983680111}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 1.7374800443649292,
 'eval_accuracy': {'accuracy': 0.7732267732267732},
 'eval_f1': {'f1': 0.7229582983680111},
 'eval_runtime': 17.229,
 'eval_samples_per_second': 58.1,
 'eval_steps_per_second': 3.657,
 'epoch': 2.0}

In [ ]:
save_dir="./model_distill/best_model"
model.save_pretrained(save_dir)
model_best = AutoModelForSequenceClassification.from_pretrained(save_dir)

In [ ]:
import torch
predictions = []
for example in val_dataset:
    #print(example)
    inp=tokenizer(example["text"], return_tensors='pt')
    with torch.no_grad():
        outputs = model_best(**inp)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predicted_labels = torch.argmax(probabilities, dim=1).tolist()
        predictions.extend(predicted_labels)